In [ ]:
#PATE VOTE SMS SPAM

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

# Đọc dữ liệu
df = pd.read_csv('/content/sample_data/spam.csv')

# Mã hóa nhãn: spam=1, ham=0
df['target'] = df['label'].map({'spam': 1, 'ham': 0})

# b) Tách dữ liệu huấn luyện và kiểm tra (70% train, 30% test)
df_train = df.sample(frac=0.7, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

y_train = df_train['target'].values
y_test = df_test['target'].values
X_train = df_train['sms'].values
X_test = df_test['sms'].values

# Tạo Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word

# Xử lý văn bản: chuyển văn bản thành các chỉ số
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding các chuỗi văn bản để có chiều dài cố định
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')

# Tham số cho mô hình
laenge_pads = 20
anz_woerter = len(word_dict)  # Lấy số lượng từ trong từ điển

# Định nghĩa mô hình LSTM
def create_lstm_model():
    lstm_model = Sequential()
    lstm_model.add(Embedding(input_dim=anz_woerter + 1, output_dim=20, input_length=laenge_pads))
    lstm_model.add(LSTM(400))
    lstm_model.add(Dense(1, activation='sigmoid'))
    lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return lstm_model

# b) Chia dữ liệu huấn luyện thành n client
n_clients = 5  # Số lượng client
train_data_splits_X = np.array_split(X_train_pad, n_clients)
train_data_splits_y = np.array_split(y_train, n_clients)

# Huấn luyện mô hình cho mỗi client
client_models = []
for i in range(n_clients):
    X_client = train_data_splits_X[i]
    y_client = train_data_splits_y[i]

    client_model = create_lstm_model()
    client_model.fit(X_client, y_client, epochs=10, batch_size=64)
    client_models.append(client_model)

# c) Dùng 2/3 dữ liệu kiểm tra để gửi tới các mô hình client và dự đoán
split_index = len(X_test_pad) // 3  # Tính chỉ số cắt để chia dữ liệu test thành 2/3 và 1/3
X_test_2_3 = X_test_pad[:2 * split_index]
y_test_2_3 = y_test[:2 * split_index]
X_test_1_3 = X_test_pad[2 * split_index:]
y_test_1_3 = y_test[2 * split_index:]

client_predictions = []
for model in client_models:
    predictions = model.predict(X_test_2_3)
    client_predictions.append(predictions)


print(client_predictions)
# d) Sử dụng hard vote để xác định nhãn cuối cùng
def hard_vote(predictions):
    print("HARD VOTE")
    predictions = np.array(predictions)
    vote = np.round(np.mean(predictions, axis=0))
    return vote

final_predictions = hard_vote(client_predictions)

# e) Sử dụng bộ dữ liệu đã gán nhãn để huấn luyện lại mô hình (chỉ dùng X_test_2_3 để huấn luyện)
print("Student Train")
final_model = create_lstm_model()
final_model.fit(X_test_2_3, final_predictions.flatten(), epochs=10, batch_size=64)

# f) Đánh giá độ chính xác trên 1/3 dữ liệu kiểm tra còn lại
print("Student Test")
loss, accuracy = final_model.evaluate(X_test_1_3, y_test_1_3, verbose=1)
print(f"Final test accuracy on remaining 1/3: {accuracy * 100:.2f}%")


(3900, 3) (1672, 3)
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.7392 - loss: 0.5737
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.8733 - loss: 0.3262
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.9097 - loss: 0.1720
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - accuracy: 0.9779 - loss: 0.0792
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.9916 - loss: 0.0488
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.9944 - loss: 0.0242
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 143ms/step - accuracy: 0.9948 - loss: 0.0263
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - accuracy: 0.9958 - loss: 0.0180
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 199ms/step - accuracy: 0.9995 - loss: 0.0030
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - accuracy: 1.0000 - loss: 0.0027
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 150ms/step - accuracy: 0.8420 - loss: 0.5351
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 

In [ ]:
# ============================================
# PATE VOTE SMS SPAM - SECURE AGGREGATION
# ============================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

# =======================
# 0. HÀM HỖ TRỢ
# =======================

def load_sms_data(path):
    print("=== 1. LOAD DATA ===")
    # Thử một vài encoding phổ biến
    for enc in ["utf-8", "latin-1", "ISO-8859-1"]:
        try:
            print(f"Đang thử đọc file với encoding='{enc}' ...")
            df_ = pd.read_csv(path, encoding=enc)
            print(f"Đọc thành công với encoding='{enc}'")
            break
        except UnicodeDecodeError as e:
            print(f"  -> Lỗi với encoding='{enc}': {e}")
            df_ = None

    if df_ is None:
        raise ValueError("Không đọc được file CSV với các encoding thử nghiệm.")

    print("5 dòng đầu của dữ liệu:")
    print(df_.head())

    # Chuẩn hóa tên cột: label và sms/message
    label_col_candidates = ["label", "Label", "v1", "category", "Category"]
    text_col_candidates  = ["sms", "SMS", "message", "Message", "v2", "text", "Text"]

    label_col = None
    text_col  = None

    for c in label_col_candidates:
        if c in df_.columns:
            label_col = c
            break

    for c in text_col_candidates:
        if c in df_.columns:
            text_col = c
            break

    if label_col is None or text_col is None:
        raise ValueError(f"Không tìm thấy cột label/text trong file. Columns: {df_.columns}")

    print(f"Đã chọn cột label = '{label_col}', cột text = '{text_col}'")

    df = df_[[label_col, text_col]].copy()
    df.columns = ["label", "sms"]  # chuẩn tên cột

    # Chuẩn hóa nhãn: spam=1, ham=0
    print("=== 2. CHUẨN HOÁ NHÃN ===")
    if set(df["label"].unique()) <= {"spam", "ham"}:
        df["target"] = df["label"].map({"spam": 1, "ham": 0})
    else:
        df["label_str"] = df["label"].astype(str).str.lower()
        df["target"] = df["label_str"].map(
            {"spam": 1, "ham": 0, "1": 1, "0": 0}
        )
        if df["target"].isna().any():
            raise ValueError("Không map được toàn bộ nhãn spam/ham -> 1/0, kiểm tra dữ liệu!")

    print("Phân bố nhãn:")
    print(df["target"].value_counts())

    return df


# =======================
# 1. MÔ HÌNH LSTM
# =======================

def create_lstm_model(vocab_size, max_len):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size + 1, output_dim=20, input_length=max_len))
    model.add(LSTM(128))  # nhẹ hơn 400 một chút cho demo
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# =======================
# 2. THAM SỐ PATE
# =======================


P = 10000019
G = 2
GAMMA = 2

def discrete_log_bruteforce(g, h, p, max_log):
    """
    Tìm S sao cho g^S ≡ h (mod p), với 0 <= S <= max_log.
    Brute-force: chỉ nên dùng demo / max_log nhỏ.
    """
    cur = 1
    for s in range(max_log + 1):
        if cur == h:
            return s
        cur = (cur * g) % p
    raise ValueError("Không tìm được discrete log trong khoảng cho phép")


# =======================
# 3. HARD VOTE SECURE AGGREGATION
# =======================

def hard_vote_secure_agg(predictions, p=P, g=G, gamma=GAMMA):
    """
    predictions: list length = n_clients,
                 mỗi phần tử là mảng (n_samples, 1) hoặc (n_samples,)
                 -> xác suất spam (output sigmoid) của từng teacher.

    Trả về:
        labels: mảng (n_samples, 1) nhãn 0/1.
    """
    print("\n=== 6. HARD VOTE - SECURE AGGREGATION (PATE style) ===")

    # preds: (n_clients, n_samples)
    preds_list = [np.array(pr).reshape(-1) for pr in predictions]
    preds = np.stack(preds_list, axis=0)  # (n_clients, n_samples)

    n_clients, n_samples = preds.shape
    scale = 10 ** gamma
    max_sum = n_clients * scale  # O_i ∈ [0,1] -> \tilde{O}_i ∈ [0, scale]

    print(f"Số client    : {n_clients}")
    print(f"Số mẫu vote  : {n_samples}")
    print(f"GAMMA        : {gamma} -> scale = {scale}")
    print(f"max_sum (giới hạn tìm log): {max_sum}")

    # ---- Phase 1: Initialization ----
    print("Phase 1: Khởi tạo khoá cho từng teacher")

    # Mỗi teacher có khóa riêng x_i, y_i
    x = np.random.randint(1, p - 1, size=n_clients, dtype=np.int64)
    y = np.random.randint(1, p - 1, size=n_clients, dtype=np.int64)

    X_i = [pow(g, int(x_i), p) for x_i in x]
    Y_i = [pow(g, int(y_i), p) for y_i in y]

    X = 1
    Y = 1
    for Xi in X_i:
        X = (X * Xi) % p
    for Yi in Y_i:
        Y = (Y * Yi) % p

    print(f"Giá trị X (tổng hợp): {X}")
    print(f"Giá trị Y (tổng hợp): {Y}")

    # ---- Phase 2: Secure computation ----

    # Bước 1: lượng tử hoá dự đoán
    print("Phase 2 - Step 1: Lượng tử hoá dự đoán của từng teacher")
    quantized = (preds * scale).round().astype(int)  # (n_clients, n_samples)

    # Bước 2: mỗi teacher mã hoá dự đoán của mình
    print("Phase 2 - Step 2: Mã hoá dự đoán của từng teacher")
    V_all = []
    for i in range(n_clients):
        # X^{y_i} / Y^{x_i} (mod p)
        X_y = pow(X, int(y[i]), p)
        Y_x = pow(Y, int(x[i]), p)
        Y_x_inv = pow(Y_x, p - 2, p)  # nghịch đảo modulo p (Fermat)
        key_factor = (X_y * Y_x_inv) % p

        V_i = []
        for s in range(n_samples):
            g_pow = pow(g, int(quantized[i, s]), p)
            V_is = (key_factor * g_pow) % p
            V_i.append(V_is)
        V_all.append(np.array(V_i, dtype=np.int64))

    V_all = np.stack(V_all, axis=0)  # (n_clients, n_samples)

    # Bước 3: Aggregator nhân các vector V_i
    print("Phase 2 - Step 3: Aggregator nhân các bản mã V_i lại với nhau")
    V_agg = np.ones(n_samples, dtype=np.int64)
    for i in range(n_clients):
        V_agg = (V_agg * V_all[i]) % p

    # Bước 4: Giải discrete log để tìm tổng lượng tử S^(j)
    print("Phase 2 - Step 4: Giải discrete log để khôi phục tổng lượng tử S^(j)")
    S = np.zeros(n_samples, dtype=np.int64)
    for s in range(n_samples):
        S[s] = discrete_log_bruteforce(g, int(V_agg[s]), p, max_sum)
        if s < 3:  # in demo một vài mẫu đầu
            print(f"  Mẫu {s} - V_agg={V_agg[s]}, S={S[s]}")

    # Bước 5: Khôi phục tổng thực O_sum^(j) = S^(j) / 10^gamma
    print("Phase 2 - Step 5: Chia ngược scale để lấy tổng xác suất")
    O_sum = S.astype(float) / scale  # ∑_i O_i^(j)

    # Trung bình xác suất và hard vote
    avg_prob = O_sum / n_clients
    labels = (avg_prob >= 0.5).astype(int).reshape(-1, 1)

    print("Một vài mẫu demo (sample 0-2):")
    for idx in range(min(3, n_samples)):
        print(f"  Sample {idx}:")
        print(f"    Dự đoán từng teacher: {preds[:, idx]}")
        print(f"    Tổng lượng tử S      : {S[idx]}")
        print(f"    Tổng xác suất ~      : {O_sum[idx]}")
        print(f"    Xác suất TB          : {avg_prob[idx]}")
        print(f"    Nhãn cuối cùng       : {labels[idx, 0]}")

    return labels


# =======================
# 4. MAIN FLOW
# =======================

# Đường dẫn file CSV của bạn
CSV_PATH = "/content/sample_data/spam.csv"   # đổi lại nếu cần (spam_1.csv, ...)

# 4.1 Load và chuẩn dữ liệu
df = load_sms_data(CSV_PATH)

# 4.2 Tách train/test (70% / 30%)
print("\n=== 3. CHIA TRAIN / TEST ===")
df_train = df.sample(frac=0.7, random_state=11)
df_test = df.drop(df_train.index)
print("Kích thước train:", df_train.shape)
print("Kích thước test :", df_test.shape)

y_train = df_train["target"].values
y_test = df_test["target"].values
X_train = df_train["sms"].values
X_test = df_test["sms"].values

# 4.3 Tokenizer & padding
print("\n=== 4. TOKENIZE & PADDING ===")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word
vocab_size = len(word_dict)
print("Số lượng từ trong vocab:", vocab_size)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

MAX_LEN = 20
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

print("Shape X_train_pad:", X_train_pad.shape)
print("Shape X_test_pad :", X_test_pad.shape)

# 4.4 Chia dữ liệu train thành n client
print("\n=== 5. CHIA DỮ LIỆU CHO CÁC TEACHER (CLIENT) ===")
n_clients = 5
train_data_splits_X = np.array_split(X_train_pad, n_clients)
train_data_splits_y = np.array_split(y_train, n_clients)

for i in range(n_clients):
    print(f"  Client {i+1}: X={train_data_splits_X[i].shape}, y={train_data_splits_y[i].shape}")

# 4.5 Huấn luyện mô hình cho mỗi client (teacher)
print("\n=== 6. TRAIN CÁC TEACHER MODEL ===")
client_models = []
for i in range(n_clients):
    print(f"\n--- Training client {i+1}/{n_clients} ---")
    X_client = train_data_splits_X[i]
    y_client = train_data_splits_y[i]

    model = create_lstm_model(vocab_size=vocab_size, max_len=MAX_LEN)
    model.fit(X_client, y_client, epochs=10, batch_size=64, verbose=1)  # epochs nhỏ cho nhanh
    client_models.append(model)

# 4.6 Chia tập test: 2/3 cho teacher vote, 1/3 cho student test
print("\n=== 7. CHIA TẬP TEST: 2/3 DÙNG ĐỂ VOTE, 1/3 DÙNG ĐỂ TEST STUDENT ===")
split_index = len(X_test_pad) // 3
X_test_2_3 = X_test_pad[:2 * split_index]
y_test_2_3 = y_test[:2 * split_index]
X_test_1_3 = X_test_pad[2 * split_index:]
y_test_1_3 = y_test[2 * split_index:]

print("X_test_2_3:", X_test_2_3.shape, "y_test_2_3:", y_test_2_3.shape)
print("X_test_1_3:", X_test_1_3.shape, "y_test_1_3:", y_test_1_3.shape)

# 4.7 Cho các teacher dự đoán trên 2/3 test
print("\n=== 8. TEACHER PREDICTIONS TRÊN 2/3 TẬP TEST ===")
client_predictions = []
for i, model in enumerate(client_models):
    print(f"  Dự đoán từ client {i+1} ...")
    preds = model.predict(X_test_2_3, verbose=0)
    client_predictions.append(preds)

print("Số mảng dự đoán nhận được:", len(client_predictions))
print("Shape dự đoán từ client 1:", client_predictions[0].shape)

# 4.8 Hard vote theo giao thức secure aggregation
final_predictions = hard_vote_secure_agg(client_predictions)

# 4.9 Huấn luyện student model với dữ liệu đã gán nhãn
print("\n=== 9. STUDENT TRAIN ===")
student_model = create_lstm_model(vocab_size=vocab_size, max_len=MAX_LEN)
student_model.fit(X_test_2_3, final_predictions.flatten(), epochs=10, batch_size=64, verbose=1)

# 4.10 Đánh giá student model trên 1/3 test còn lại
print("\n=== 10. STUDENT TEST ===")
loss, accuracy = student_model.evaluate(X_test_1_3, y_test_1_3, verbose=1)
print(f"\nFinal test accuracy on remaining 1/3: {accuracy * 100:.2f}%")


=== 1. LOAD DATA ===
Đang thử đọc file với encoding='utf-8' ...
Đọc thành công với encoding='utf-8'
5 dòng đầu của dữ liệu:
  label                                                sms
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
Đã chọn cột label = 'label', cột text = 'sms'
=== 2. CHUẨN HOÁ NHÃN ===
Phân bố nhãn:
target
0    4825
1     747
Name: count, dtype: int64

=== 3. CHIA TRAIN / TEST ===
Kích thước train: (3900, 3)
Kích thước test : (1672, 3)

=== 4. TOKENIZE & PADDING ===
Số lượng từ trong vocab: 7376
Shape X_train_pad: (3900, 20)
Shape X_test_pad : (1672, 20)

=== 5. CHIA DỮ LIỆU CHO CÁC TEACHER (CLIENT) ===
  Client 1: X=(780, 20), y=(780,)
  Client 2: X=(780, 20), y=(780,)
  Client 3: X=(780, 20), y=(780,)
  Client 4: X=(780, 20),

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8688 - loss: 0.6013
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8897 - loss: 0.2251
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9211 - loss: 0.1421
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9847 - loss: 0.0988
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9916 - loss: 0.0343
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9933 - loss: 0.0267
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9981 - loss: 0.0098
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9992 - loss: 0.0062
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9964 - loss: 0.0043
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 6.8825e-04

--- Training client 2/5 ---
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.7379 - loss: 0.6141
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 3

In [ ]:
# ============================================
# PATE VOTE MNIST với Secure Aggregation
# ============================================

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

np.random.seed(42)
tf.random.set_seed(42)

# =======================
# 0. THAM SỐ PATE
# =======================


P = 10000019
G = 2
GAMMA = 2

def build_dlog_table(g, p, max_log):
    """
    Build bảng tra discrete log: value -> exponent
    Tính g^s mod p cho s=0..max_log và lưu trong dict.
    """
    table = {}
    val = 1
    for s in range(max_log + 1):
        if val not in table:   # lưu exp nhỏ nhất
            table[val] = s
        val = (val * g) % p
    return table


# =======================
# 1. ĐỌC & TIỀN XỬ LÝ MNIST
# =======================

print("=== 1. ĐỌC DỮ LIỆU MNIST ===")
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("  X_train shape:", X_train.shape)
print("  X_test  shape:", X_test.shape)

# Tiền xử lý dữ liệu
print("\n=== 2. TIỀN XỬ LÝ DỮ LIỆU ===")
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255  # CNN input
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255

print("  Sau reshape:")
print("  X_train shape:", X_train.shape)
print("  X_test  shape:", X_test.shape)

num_classes = 10

# Tách tập kiểm tra thành 2 phần: 2/3 cho PATE/Student train, 1/3 cho đánh giá
print("\n=== 3. CHIA TẬP TEST: 2/3 DÙNG PATE, 1/3 DÙNG TEST STUDENT ===")
split_index = len(X_test) // 3
X_test_2_3 = X_test[:2 * split_index]
y_test_2_3 = y_test[:2 * split_index]
X_test_1_3 = X_test[2 * split_index:]
y_test_1_3 = y_test[2 * split_index:]

print("  X_test_2_3:", X_test_2_3.shape, "y_test_2_3:", y_test_2_3.shape)
print("  X_test_1_3:", X_test_1_3.shape, "y_test_1_3:", y_test_1_3.shape)


# =======================
# 2. CHIA TRAIN CHO CÁC CLIENT (TEACHER)
# =======================

print("\n=== 4. CHIA DỮ LIỆU HUẤN LUYỆN CHO CÁC TEACHER ===")
n_clients = 5  # Số lượng client
train_data_splits_X = np.array_split(X_train, n_clients)
train_data_splits_y = np.array_split(y_train, n_clients)

for i in range(n_clients):
    print(f"  Client {i+1}: X={train_data_splits_X[i].shape}, y={train_data_splits_y[i].shape}")


# =======================
# 3. ĐỊNH NGHĨA MÔ HÌNH CNN
# =======================

def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model


# =======================
# 4. TRAIN TEACHER MODELS
# =======================

print("\n=== 5. HUẤN LUYỆN MỖI TEACHER (CLIENT) ===")
client_models = []
for i in range(n_clients):
    print(f"\n--- Training client {i+1}/{n_clients} ---")
    X_client = train_data_splits_X[i]
    y_client = train_data_splits_y[i]

    client_model = create_cnn_model()
    client_model.fit(X_client, y_client, epochs=10, batch_size=64, verbose=1)
    client_models.append(client_model)


# =======================
# 5. TEACHER PREDICTIONS TRÊN 2/3 TẬP TEST
# =======================

print("\n=== 6. TEACHER PREDICTIONS TRÊN 2/3 TẬP TEST ===")
client_predictions = []
for i, model in enumerate(client_models):
    print(f"  Dự đoán từ client {i+1} ...")
    predictions = model.predict(X_test_2_3, verbose=0)  # (n_samples, 10)
    client_predictions.append(predictions)

print("  Số mảng dự đoán nhận được:", len(client_predictions))
print("  Shape dự đoán từ client 1:", client_predictions[0].shape)


# =======================
# 6. HARD VOTE VỚI SECURE AGGREGATION (PATE)
# =======================

def hard_vote_secure_agg_multiclass(predictions, num_classes, p=P, g=G, gamma=GAMMA):
    """
    predictions: list length = n_clients,
                 mỗi phần tử là mảng (n_samples, num_classes)
                 -> xác suất softmax của từng teacher.

    Trả về:
        labels: mảng (n_samples,) chứa nhãn 0..num_classes-1.
    """
    print("\n=== 7. HARD VOTE - SECURE AGGREGATION (PATE, MULTI-CLASS) ===")

    preds_list = [np.array(pr) for pr in predictions]
    preds = np.stack(preds_list, axis=0)  # (n_clients, n_samples, num_classes)

    n_clients, n_samples, num_classes2 = preds.shape
    assert num_classes2 == num_classes, "num_classes không khớp!"

    scale = 10 ** gamma
    max_sum = n_clients * scale  # tối đa tổng lượng tử

    print(f"  Số client       : {n_clients}")
    print(f"  Số mẫu vote     : {n_samples}")
    print(f"  Số lớp          : {num_classes}")
    print(f"  GAMMA           : {gamma} -> scale = {scale}")
    print(f"  max_sum (giới hạn discrete log): {max_sum}")

    # -------- Phase 1: Initialization --------
    print("  Phase 1: Khởi tạo khóa cho từng teacher & từng class")

    # x_i^(j), y_i^(j): shape (n_clients, num_classes)
    x = np.random.randint(1, p - 1, size=(n_clients, num_classes), dtype=np.int64)
    y = np.random.randint(1, p - 1, size=(n_clients, num_classes), dtype=np.int64)

    # Tính X^(j), Y^(j) cho mỗi class j
    X = np.ones(num_classes, dtype=np.int64)
    Y = np.ones(num_classes, dtype=np.int64)

    for j in range(num_classes):
        for i in range(n_clients):
            X[j] = (X[j] * pow(g, int(x[i, j]), p)) % p
            Y[j] = (Y[j] * pow(g, int(y[i, j]), p)) % p

    print("  Ví dụ X[0], Y[0]:", X[0], Y[0])

    # -------- Phase 2: Secure computation --------

    # Bước 1: lượng tử hóa dự đoán
    print("  Phase 2 - Step 1: Lượng tử hóa dự đoán")
    quantized = (preds * scale).round().astype(int)  # (n_clients, n_samples, num_classes)

    # Bước 2: Tính key_factor[i, j] = X^(j)^{y_i^(j)} * (Y^(j)^{x_i^(j)})^{-1} mod p
    print("  Phase 2 - Step 2: Tính key_factor cho từng client & lớp")
    key_factor = np.zeros((n_clients, num_classes), dtype=np.int64)
    for i in range(n_clients):
        for j in range(num_classes):
            X_y = pow(int(X[j]), int(y[i, j]), p)
            Y_x = pow(int(Y[j]), int(x[i, j]), p)
            Y_x_inv = pow(Y_x, p - 2, p)  # nghịch đảo modulo p (Fermat)
            key_factor[i, j] = (X_y * Y_x_inv) % p

    # Bước 3: Mã hóa & aggregate
    print("  Phase 2 - Step 3: Mã hóa & nhân các bản mã V_i^(j)")
    V_agg = np.ones((n_samples, num_classes), dtype=np.int64)

    for i in range(n_clients):
        for s in range(n_samples):
            for j in range(num_classes):
                g_pow = pow(g, int(quantized[i, s, j]), p)
                V_i_s_j = (key_factor[i, j] * g_pow) % p
                V_agg[s, j] = (V_agg[s, j] * V_i_s_j) % p

    # Bước 4: Giải discrete log để khôi phục S^(k,j)
    print("  Phase 2 - Step 4: Giải discrete log để khôi phục S^(k,j)")
    print("    -> Xây bảng tra discrete log một lần cho g^s (s=0..max_sum)")
    dlog_table = build_dlog_table(g, p, max_sum)

    S = np.zeros((n_samples, num_classes), dtype=np.int64)
    for s in range(n_samples):
        for j in range(num_classes):
            v_val = int(V_agg[s, j])
            if v_val not in dlog_table:
                raise ValueError(f"Không tìm thấy discrete log cho V_agg[{s},{j}]={v_val}")
            S[s, j] = dlog_table[v_val]
        if s < 2:  # in demo 2 mẫu đầu
            print(f"    Mẫu {s} - S[{s}, :]:", S[s, :])

    # Bước 5: Khôi phục tổng xác suất & hard vote
    print("  Phase 2 - Step 5: Chia ngược scale, lấy tổng & argmax")

    O_sum = S.astype(float) / scale       # (n_samples, num_classes) ~ ∑ O_i^(k,j)
    O_avg = O_sum / n_clients            # Trung bình xác suất trên các teacher
    labels = np.argmax(O_avg, axis=1)    # Hard vote bằng argmax

    print("  Một vài mẫu demo (sample 0-2):")
    for idx in range(min(3, n_samples)):
        print(f"    Sample {idx}:")
        print(f"      Dự đoán teacher 0: {preds[0, idx]}")
        print(f"      Tổng O_sum      : {O_sum[idx]}")
        print(f"      Trung bình O_avg: {O_avg[idx]}")
        print(f"      Nhãn cuối cùng  : {labels[idx]}")

    return labels


# GỌI HÀM HARD VOTE SECURE AGGREGATION
final_predictions = hard_vote_secure_agg_multiclass(
    client_predictions,
    num_classes=num_classes,
    p=P,
    g=G,
    gamma=GAMMA
)

print("\n  final_predictions shape:", final_predictions.shape)
print("  10 nhãn đầu:", final_predictions[:10])
print("  10 nhãn thực (y_test_2_3):", y_test_2_3[:10])


# =======================
# 7. HUẤN LUYỆN STUDENT MODEL
# =======================

print("\n=== 8. HUẤN LUYỆN STUDENT MODEL TRÊN NHÃN PATE ===")
final_model = create_cnn_model()
final_model.fit(X_test_2_3, final_predictions, epochs=10, batch_size=64, verbose=1)

# =======================
# 8. ĐÁNH GIÁ TRÊN 1/3 TẬP TEST CÒN LẠI
# =======================

print("\n=== 9. ĐÁNH GIÁ STUDENT MODEL TRÊN 1/3 TẬP TEST CÒN LẠI ===")
loss, accuracy = final_model.evaluate(X_test_1_3, y_test_1_3, verbose=1)
print(f"\nFinal test accuracy on remaining 1/3: {accuracy * 100:.2f}%")


=== 1. ĐỌC DỮ LIỆU MNIST ===
  X_train shape: (60000, 28, 28)
  X_test  shape: (10000, 28, 28)

=== 2. TIỀN XỬ LÝ DỮ LIỆU ===
  Sau reshape:
  X_train shape: (60000, 28, 28, 1)
  X_test  shape: (10000, 28, 28, 1)

=== 3. CHIA TẬP TEST: 2/3 DÙNG PATE, 1/3 DÙNG TEST STUDENT ===
  X_test_2_3: (6666, 28, 28, 1) y_test_2_3: (6666,)
  X_test_1_3: (3334, 28, 28, 1) y_test_1_3: (3334,)

=== 4. CHIA DỮ LIỆU HUẤN LUYỆN CHO CÁC TEACHER ===
  Client 1: X=(12000, 28, 28, 1), y=(12000,)
  Client 2: X=(12000, 28, 28, 1), y=(12000,)
  Client 3: X=(12000, 28, 28, 1), y=(12000,)
  Client 4: X=(12000, 28, 28, 1), y=(12000,)
  Client 5: X=(12000, 28, 28, 1), y=(12000,)

=== 5. HUẤN LUYỆN MỖI TEACHER (CLIENT) ===

--- Training client 1/5 ---
Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.7385 - loss: 0.8826
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9599 - loss: 0.1461
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9698 - loss: 0.1005
Epoch 4/10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Đường dẫn đến thư mục Research
path_to_research = '/content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset'

# Kiểm tra xem thư mục có tồn tại không
if os.path.exists(path_to_research):
    print(f"Found 'Research' folder at: {path_to_research}")
else:
    print("The 'Research' folder does not exist.")

Mounted at /content/drive
Found 'Research' folder at: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset


In [ ]:
path = "/content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset"

images = []
labels = []
class_names = []

i = 0
for folder_name in os.listdir(path):
    class_folder = os.path.join(path, folder_name)

    # Bỏ qua nếu không phải thư mục (phòng khi có file lẻ)
    if not os.path.isdir(class_folder):
        print("Bỏ qua (không phải thư mục class):", class_folder)
        continue

    # Đường dẫn tới thư mục images bên trong
    images_folder = os.path.join(class_folder, "images")
    if not os.path.isdir(images_folder):
        print("  Không thấy thư mục 'images' trong:", class_folder)
        continue

    print(f"Đang đọc class {i} - folder: {folder_name} (images_folder: {images_folder})")
    class_names.append(folder_name)

    # Duyệt qua từng file ảnh trong /images
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)

        # Nếu là thư mục con thì bỏ qua
        if os.path.isdir(image_path):
            print("  Bỏ qua subfolder trong images:", image_path)
            continue

        img = cv2.imread(image_path)
        if img is None:
            print("  Không đọc được (bỏ qua):", image_path)
            continue

        # Chuyển grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Resize
        img = cv2.resize(img, (224, 224))
        # Chuẩn hóa
        img = img.astype("float32") / 255.0

        images.append(img)
        labels.append(i)

    i += 1

images = np.array(images)
labels = np.array(labels)

print("Tổng số ảnh đọc được:", images.shape)
print("Tổng số nhãn        :", labels.shape)
print("Các class tìm được  :", class_names)

# Nếu dùng CNN 2D: thêm dimension channels
images_cnn = images.reshape(-1, 224, 224, 1)
print("Shape ảnh cho CNN   :", images_cnn.shape)


Đang đọc class 0 - folder: Viral Pneumonia (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/Viral Pneumonia/images)
Đang đọc class 1 - folder: COVID (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/COVID/images)
Đang đọc class 2 - folder: Lung_Opacity (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/Lung_Opacity/images)
Đang đọc class 3 - folder: Normal (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/Normal/images)
Tổng số ảnh đọc được: (21165, 224, 224)
Tổng số nhãn        : (21165,)
Các class tìm được  : ['Viral Pneumonia', 'COVID', 'Lung_Opacity', 'Normal']
Shape ảnh cho CNN   : (21165, 224, 224, 1)


In [ ]:
import os
import cv2
import numpy as np
from keras import layers, models, Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

# ========= 0. THAM SỐ SECURE AGGREGATION (DEMO) =========
# Lưu ý: Đây là tham số DEMO, không dùng cho crypto thực tế

P = 10000019   # p > n_clients * 10^gamma
G = 2          # giả sử 2 là generator phù hợp
GAMMA = 2      # giữ 2 chữ số sau dấu phẩy -> scale = 100

def build_dlog_table(g, p, max_log):
    """
    Xây bảng tra discrete log: value -> exponent
    value = g^s mod p, với s = 0..max_log
    """
    table = {}
    val = 1
    for s in range(max_log + 1):
        if val not in table:   # lưu exp nhỏ nhất
            table[val] = s
        val = (val * g) % p
    return table

def hard_vote_secure_agg_multiclass(predictions, num_classes, p=P, g=G, gamma=GAMMA):
    """
    predictions: list length = n_clients,
                 mỗi phần tử là mảng (n_samples, num_classes)
                 -> xác suất softmax của từng teacher.

    Trả về:
        labels: mảng (n_samples,) chứa nhãn 0..num_classes-1 (hard vote).
    """
    print("\n=== HARD VOTE - SECURE AGGREGATION (PATE, MULTI-CLASS) ===")

    preds_list = [np.array(pr) for pr in predictions]
    preds = np.stack(preds_list, axis=0)  # (n_clients, n_samples, num_classes)

    n_clients, n_samples, num_classes2 = preds.shape
    assert num_classes2 == num_classes, "num_classes không khớp!"

    scale = 10 ** gamma
    max_sum = n_clients * scale  # tối đa tổng lượng tử hóa cho mỗi class

    print(f"  Số client       : {n_clients}")
    print(f"  Số mẫu vote     : {n_samples}")
    print(f"  Số lớp (class)  : {num_classes}")
    print(f"  GAMMA           : {gamma} -> scale = {scale}")
    print(f"  max_sum (log max): {max_sum}")

    # -------- Phase 1: Initialization --------
    print("  Phase 1: Khởi tạo khóa cho từng teacher & từng class")

    # x_i^(j), y_i^(j): shape (n_clients, num_classes)
    x = np.random.randint(1, p - 1, size=(n_clients, num_classes), dtype=np.int64)
    y = np.random.randint(1, p - 1, size=(n_clients, num_classes), dtype=np.int64)

    # Tính X^(j), Y^(j) cho mỗi class j
    X = np.ones(num_classes, dtype=np.int64)
    Y = np.ones(num_classes, dtype=np.int64)

    for j in range(num_classes):
        for i in range(n_clients):
            X[j] = (X[j] * pow(g, int(x[i, j]), p)) % p
            Y[j] = (Y[j] * pow(g, int(y[i, j]), p)) % p

    print("  Ví dụ X[0], Y[0]:", X[0], Y[0])

    # -------- Phase 2: Secure computation --------

    # Bước 1: lượng tử hóa dự đoán
    print("  Phase 2 - Step 1: Lượng tử hóa dự đoán")
    quantized = (preds * scale).round().astype(int)  # (n_clients, n_samples, num_classes)

    # Bước 2: Tính key_factor[i, j] = X^(j)^{y_i^(j)} * (Y^(j)^{x_i^(j)})^{-1} mod p
    print("  Phase 2 - Step 2: Tính key_factor cho từng client & class")
    key_factor = np.zeros((n_clients, num_classes), dtype=np.int64)
    for i in range(n_clients):
        for j in range(num_classes):
            X_y = pow(int(X[j]), int(y[i, j]), p)
            Y_x = pow(int(Y[j]), int(x[i, j]), p)
            Y_x_inv = pow(Y_x, p - 2, p)  # nghịch đảo modulo p
            key_factor[i, j] = (X_y * Y_x_inv) % p

    # Bước 3: Mã hóa & aggregate
    print("  Phase 2 - Step 3: Mã hóa & nhân các bản mã V_i^(j)")
    V_agg = np.ones((n_samples, num_classes), dtype=np.int64)

    for i in range(n_clients):
        for s in range(n_samples):
            for j in range(num_classes):
                g_pow = pow(g, int(quantized[i, s, j]), p)
                V_i_s_j = (key_factor[i, j] * g_pow) % p
                V_agg[s, j] = (V_agg[s, j] * V_i_s_j) % p

    # Bước 4: Giải discrete log để khôi phục S^(k,j)
    print("  Phase 2 - Step 4: Giải discrete log để khôi phục S^(k,j)")
    print("    -> Xây bảng tra discrete log cho g^s (s=0..max_sum)")
    dlog_table = build_dlog_table(g, p, max_sum)

    S = np.zeros((n_samples, num_classes), dtype=np.int64)
    for s in range(n_samples):
        for j in range(num_classes):
            v_val = int(V_agg[s, j])
            if v_val not in dlog_table:
                raise ValueError(f"Không tìm thấy discrete log cho V_agg[{s},{j}]={v_val}")
            S[s, j] = dlog_table[v_val]
        if s < 2:  # in demo 2 mẫu đầu
            print(f"    Mẫu {s} - S[{s}, :]:", S[s, :])

    # Bước 5: Khôi phục tổng & hard vote
    print("  Phase 2 - Step 5: Chia ngược scale, lấy tổng & argmax")
    O_sum = S.astype(float) / scale       # ∑ O_i^(k,j)
    O_avg = O_sum / n_clients             # Trung bình xác suất
    labels = np.argmax(O_avg, axis=1)     # Hard vote

    print("  Một vài mẫu demo (sample 0-2):")
    for idx in range(min(3, n_samples)):
        print(f"    Sample {idx}:")
        print(f"      Dự đoán teacher 0: {preds[0, idx]}")
        print(f"      Tổng O_sum      : {O_sum[idx]}")
        print(f"      Trung bình O_avg: {O_avg[idx]}")
        print(f"      Nhãn cuối cùng  : {labels[idx]}")

    return labels


# ========= 1. LOAD ẢNH (RGB, s x s x 3) =========
s = 224  # kích thước ảnh đầu vào

path = "/content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset"

images = []
labels = []
class_names = []

i = 0
for folder_name in os.listdir(path):
    class_folder = os.path.join(path, folder_name)

    # Bỏ qua nếu không phải thư mục (phòng khi có file lẻ)
    if not os.path.isdir(class_folder):
        print("Bỏ qua (không phải thư mục class):", class_folder)
        continue

    # Đường dẫn tới thư mục images bên trong mỗi class
    images_folder = os.path.join(class_folder, "images")
    if not os.path.isdir(images_folder):
        print("  Không thấy thư mục 'images' trong:", class_folder)
        continue

    print(f"Đang đọc class {i} - folder: {folder_name} (images_folder: {images_folder})")
    class_names.append(folder_name)

    # Duyệt qua từng file ảnh trong /images
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)

        # Nếu là thư mục con thì bỏ qua
        if os.path.isdir(image_path):
            print("  Bỏ qua subfolder trong images:", image_path)
            continue

        img = cv2.imread(image_path)  # BGR
        if img is None:
            print("  Không đọc được (bỏ qua):", image_path)
            continue

        # Resize về s x s
        img = cv2.resize(img, (s, s))
        # Chuyển sang RGB cho "đẹp" (optional)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Chuẩn hóa
        img = img.astype("float32") / 255.0

        images.append(img)
        labels.append(i)

    i += 1

images = np.array(images)        # (N, s, s, 3)
labels = np.array(labels)        # (N,)

print("Tổng số ảnh đọc được:", images.shape)
print("Tổng số nhãn        :", labels.shape)
print("Các class tìm được  :", class_names)

num_classes = len(class_names)
print("Số lớp (num_classes):", num_classes)

# ========= 2. CHUẨN BỊ LABEL & SPLIT =========

label_ids = labels.copy()
labels_cat = to_categorical(label_ids, num_classes=num_classes)   # (N, num_classes)

x_train, x_test, y_train, y_test = train_test_split(
    images,
    labels_cat,
    test_size=0.3,
    random_state=42,
    stratify=label_ids
)

print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test :", x_test.shape)
print("y_test :", y_test.shape)

# ========= 3. ĐỊNH NGHĨA MODEL CNN THEO KIẾN TRÚC BẠN ĐƯA =========

def create_cnn_model(input_shape=(s, s, 3), num_classes=3):
    model = Sequential()
    model.add(layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        activation="relu",
        input_shape=input_shape,
        kernel_initializer='he_normal',
    ))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(layers.Dropout(0.25))

    # fully connected
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.2))
    # Dòng dưới là chỗ mình generalize: dùng num_classes thay vì cố định 3
    model.add(layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer='SGD',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("\n=== KIỂM TRA KIẾN TRÚC MODEL THEO BẠN ===")
tmp_model = create_cnn_model(input_shape=(s, s, 3), num_classes=num_classes)
tmp_model.summary()
del tmp_model

# ========= 4. CHIA TRAIN CHO N TEACHER =========

print("\n=== CHIA DỮ LIỆU TRAIN CHO CÁC TEACHER ===")
n_clients = 5
train_splits_x = np.array_split(x_train, n_clients)
train_splits_y = np.array_split(y_train, n_clients)

for idx in range(n_clients):
    print(f"  Teacher {idx+1}: x={train_splits_x[idx].shape}, y={train_splits_y[idx].shape}")

# ========= 5. TRAIN TEACHER MODELS =========

print("\n=== TRAIN CÁC TEACHER MODEL ===")
teacher_models = []
for i in range(n_clients):
    print(f"\n--- Training teacher {i+1}/{n_clients} ---")
    model_teacher = create_cnn_model(input_shape=(s, s, 3), num_classes=num_classes)
    model_teacher.fit(
        train_splits_x[i],
        train_splits_y[i],
        epochs=50,
        batch_size=32,
        verbose=1
    )
    teacher_models.append(model_teacher)

# ========= 6. CHIA x_test LÀM PUBLIC (2/3) & PRIVATE TEST (1/3) =========

print("\n=== CHIA x_test: 2/3 PUBLIC (PATE), 1/3 PRIVATE TEST ===")
split_idx = len(x_test) // 3
x_public = x_test[:2 * split_idx]
y_public_true = y_test[:2 * split_idx]        # chỉ để tham khảo
x_private_test = x_test[2 * split_idx:]
y_private_test = y_test[2 * split_idx:]

print("x_public      :", x_public.shape)
print("y_public_true :", y_public_true.shape)
print("x_private_test:", x_private_test.shape)
print("y_private_test:", y_private_test.shape)

# ========= 7. TEACHERS DỰ ĐOÁN TRÊN PUBLIC DATA =========

print("\n=== TEACHER PREDICTIONS TRÊN PUBLIC DATA ===")
teacher_predictions = []
for i, m in enumerate(teacher_models):
    print(f"  Dự đoán từ teacher {i+1} ...")
    preds = m.predict(x_public, verbose=0)   # (n_public, num_classes)
    teacher_predictions.append(preds)

print("  Số mảng dự đoán:", len(teacher_predictions))
print("  Shape dự đoán teacher 1:", teacher_predictions[0].shape)

# ========= 8. SECURE AGGREGATION HARD VOTE (PATE) =========

pseudo_labels_int = hard_vote_secure_agg_multiclass(
    teacher_predictions,
    num_classes=num_classes,
    p=P,
    g=G,
    gamma=GAMMA
)  # shape: (n_public,)

print("\nPseudo-labels (int) shape:", pseudo_labels_int.shape)
print("10 nhãn PATE đầu   :", pseudo_labels_int[:10])

# Chuyển sang one-hot để train student với categorical_crossentropy
pseudo_labels_cat = to_categorical(pseudo_labels_int, num_classes=num_classes)
x_private_test=x_public
y_private_test=pseudo_labels_cat

# ========= 9. TRAIN STUDENT MODEL TRÊN NHÃN PATE =========

print("\n=== TRAIN STUDENT MODEL TRÊN PUBLIC DATA + LABEL PATE ===")
student_model = create_cnn_model(input_shape=(s, s, 3), num_classes=num_classes)
student_model.fit(
    x_public,
    pseudo_labels_cat,
    epochs=20,
    batch_size=32,
    verbose=0
)

# ========= 10. EVALUATE STUDENT TRÊN PRIVATE TEST =========

print("\n=== ĐÁNH GIÁ STUDENT MODEL TRÊN 1/3 TEST CÒN LẠI ===")
loss, acc = student_model.evaluate(x_private_test, y_private_test, verbose=1)
print(f"\nFinal test accuracy on remaining 1/3: {acc * 100:.2f}%")


Đang đọc class 0 - folder: Viral Pneumonia (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/Viral Pneumonia/images)
Đang đọc class 1 - folder: COVID (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/COVID/images)
Đang đọc class 2 - folder: Lung_Opacity (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/Lung_Opacity/images)
Đang đọc class 3 - folder: Normal (images_folder: /content/drive/MyDrive/Research/SecureAI-Lab/dataset/Covid19/COVID-19_Radiography_Dataset/Normal/images)
Tổng số ảnh đọc được: (21165, 224, 224, 3)
Tổng số nhãn        : (21165,)
Các class tìm được  : ['Viral Pneumonia', 'COVID', 'Lung_Opacity', 'Normal']
Số lớp (num_classes): 4
x_train: (14815, 224, 224, 3)
y_train: (14815, 4)
x_test : (6350, 224, 224, 3)
y_test : (6350, 4)

=== KIỂM TRA KIẾN TRÚC MODEL THEO BẠN ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 51, 51, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 51, 51, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 332928)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │    21,307,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,400,964 (81.64 MB)

 Trainable params: 21,400,964 (81.64 MB)

 Non-trainable params: 0 (0.00 B)


=== CHIA DỮ LIỆU TRAIN CHO CÁC TEACHER ===
  Teacher 1: x=(2963, 224, 224, 3), y=(2963, 4)
  Teacher 2: x=(2963, 224, 224, 3), y=(2963, 4)
  Teacher 3: x=(2963, 224, 224, 3), y=(2963, 4)
  Teacher 4: x=(2963, 224, 224, 3), y=(2963, 4)
  Teacher 5: x=(2963, 224, 224, 3), y=(2963, 4)

=== TRAIN CÁC TEACHER MODEL ===

--- Training teacher 1/5 ---
Epoch 1/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.4098 - loss: 1.5548
Epoch 2/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4881 - loss: 1.0919
Epoch 3/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5464 - loss: 1.0150
Epoch 4/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5639 - loss: 0.9781
Epoch 5/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6178 - loss: 0.9016
Epoch 6/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6299 - loss: 0.8805
Epoch 7/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6479 - loss: 0.8634
Epoch 8/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accur

In [ ]:
# ========= 9. TRAIN STUDENT MODEL TRÊN NHÃN PATE =========

print("\n=== TRAIN STUDENT MODEL TRÊN PUBLIC DATA + LABEL PATE ===")
student_model = create_cnn_model(input_shape=(s, s, 3), num_classes=num_classes)
student_model.fit(
    x_public,
    pseudo_labels_cat,
    epochs=20,
    batch_size=32,
    verbose=0
)

# ========= 10. EVALUATE STUDENT TRÊN PRIVATE TEST =========

print("\n=== ĐÁNH GIÁ STUDENT MODEL TRÊN 1/3 TEST CÒN LẠI ===")
loss, acc = student_model.evaluate(x_private_test, y_private_test, verbose=1)
print(f"\nFinal test accuracy on remaining 1/3: {acc * 100:.2f}%")


=== TRAIN STUDENT MODEL TRÊN PUBLIC DATA + LABEL PATE ===

=== ĐÁNH GIÁ STUDENT MODEL TRÊN 1/3 TEST CÒN LẠI ===
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9618 - loss: 0.1144

Final test accuracy on remaining 1/3: 96.34%
